## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv(r"C:/Users/rbing/Classwork/World_Weather_Analysis/Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Arraial Do Cabo,BR,-22.9661,-42.0278,71.46,68,75,3.44,broken clouds
1,1,Longyearbyen,SJ,78.2186,15.6401,19.24,86,75,10.36,broken clouds
2,2,Butaritari,KI,3.0707,172.7902,82.56,78,37,16.33,light rain
3,3,Hithadhoo,MV,-0.6000,73.0833,82.76,72,76,11.27,broken clouds
4,4,Cape Town,ZA,-33.9258,18.4232,67.10,60,75,5.99,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Butaritari,KI,3.0707,172.7902,82.56,78,37,16.33,light rain
3,3,Hithadhoo,MV,-0.6000,73.0833,82.76,72,76,11.27,broken clouds
9,9,Saint-Philippe,RE,-21.3585,55.7679,84.47,80,52,10.63,light rain
12,12,Ullal,IN,12.8000,74.8500,86.95,74,20,5.75,few clouds
14,14,Kahului,US,20.8947,-156.4700,77.85,81,1,6.91,clear sky
15,15,Souillac,MU,-20.5167,57.5167,78.37,67,86,11.01,overcast clouds
17,17,Pitimbu,BR,-7.4706,-34.8086,82.99,71,96,9.33,overcast clouds
19,19,Alice Springs,AU,-23.7000,133.8833,89.15,18,27,10.36,scattered clouds
29,29,Trinidad,BO,-14.8333,-64.9000,81.10,64,50,5.75,scattered clouds
34,34,Batticaloa,LK,7.7102,81.6924,83.80,74,47,10.16,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                227
City                   227
Country                225
Lat                    227
Lng                    227
Max Temp               227
Humidity               227
Cloudiness             227
Wind Speed             227
Current Description    227
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_preferred_cities_df = preferred_cities_df.dropna()
clean_preferred_cities_df.count()

City_ID                225
City                   225
Country                225
Lat                    225
Lng                    225
Max Temp               225
Humidity               225
Cloudiness             225
Wind Speed             225
Current Description    225
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Butaritari,KI,82.56,light rain,3.0707,172.7902,
3,Hithadhoo,MV,82.76,broken clouds,-0.6000,73.0833,
9,Saint-Philippe,RE,84.47,light rain,-21.3585,55.7679,
12,Ullal,IN,86.95,few clouds,12.8000,74.8500,
14,Kahului,US,77.85,clear sky,20.8947,-156.4700,
15,Souillac,MU,78.37,overcast clouds,-20.5167,57.5167,
17,Pitimbu,BR,82.99,overcast clouds,-7.4706,-34.8086,
19,Alice Springs,AU,89.15,scattered clouds,-23.7000,133.8833,
29,Trinidad,BO,81.10,scattered clouds,-14.8333,-64.9000,
34,Batticaloa,LK,83.80,scattered clouds,7.7102,81.6924,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json() 
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [17]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Butaritari,KI,82.56,light rain,3.0707,172.7902,Isles Sunset Lodge
3,Hithadhoo,MV,82.76,broken clouds,-0.6000,73.0833,Scoop Guest House
9,Saint-Philippe,RE,84.47,light rain,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
12,Ullal,IN,86.95,few clouds,12.8000,74.8500,AK INTERNATIONAL LODGE
14,Kahului,US,77.85,clear sky,20.8947,-156.4700,Maui Seaside Hotel


In [18]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [19]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [20]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))